<a href="https://colab.research.google.com/github/SEOYUNJE/Lung-Image-Analysis/blob/main/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://www.researchgate.net/publication/368303422/figure/fig1/AS:11431281118333772@1675729425161/CBAM-structure-a-Convolutional-Block-Attention-Module-b-Channel-Attention-Module.png)

In [ ]:
from tensorflow.keras import *

class CAM(layers.Layer):
    def __init__(self, ratio=8):
        super().__init__()
        self.ratio = ratio
        self.gap = layers.GlobalAveragePooling2D()
        self.gmp = layers.GlobalMaxPooling2D()

    def build(self, input_shape):
        self.conv1 = layers.Conv2D(input_shape[-1]//self.ratio,
                                   kernel_size=1,
                                   strides=1, padding='same',
                                   activation='relu')
        self.conv2 = layers.Conv2D(input_shape[-1],
                                   kernel_size=1,
                                   strides=1, padding='same',
                                   activation='relu')

    def call(self, inputs):
        gap = self.gap(inputs)
        gmp = self.gmp(inputs)
        gap = layers.Reshape((1,1,gap.shape[1]))(gap)
        gmp = layers.Reshape((1,1,gmp.shape[1]))(gmp)
        gap_out = self.conv2(self.conv1(gap))
        gmp_out = self.conv2(self.conv1(gmp))

        return tf.math.sigmoid(gap_out+gmp_out)

In [ ]:

class SAM(layers.Layer):
    def __init__(self, kernel_size=3):
        super().__init__()
        self.conv1 = layers.Conv2D(64,
                                            kernel_size=kernel_size,
                                            use_bias=False,
                                            kernel_initializer='he_normal',
                                            strides=1, padding='same',
                                            activation=tf.nn.relu)
        self.conv2 = layers.Conv2D(32, kernel_size=kernel_size,
                                            use_bias=False,
                                            kernel_initializer='he_normal',
                                            strides=1, padding='same',
                                            activation=tf.nn.relu)
        self.conv3 = layers.Conv2D(16, kernel_size=kernel_size,
                                            use_bias=False,
                                            kernel_initializer='he_normal',
                                            strides=1, padding='same',
                                            activation=tf.nn.relu)
        self.conv4 = layers.Conv2D(1,
                                            kernel_size=(1, 1),
                                            use_bias=False,
                                            kernel_initializer='he_normal',
                                            strides=1, padding='same',
                                            activation=tf.math.sigmoid)


    def call(self, inputs):
        avg_out = tf.reduce_mean(inputs, axis=3)
        max_out = tf.reduce_max(inputs,  axis=3)
        x = tf.stack([avg_out, max_out], axis=3)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return self.conv4(x)

In [ ]:
class AttentionWeightedAverage2D(layers.Layer):
    def __init__(self, **kwargs):
        self.init = tf.keras.initializers.get('uniform')
        super(AttentionWeightedAverage2D, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [layers.InputSpec(ndim=4)]
        assert len(input_shape) == 4
        self.W = self.add_weight(shape=(input_shape[3], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self._trainable_weights = [self.W]
        super(AttentionWeightedAverage2D, self).build(input_shape)

    def call(self, x):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 2-dimensional weights
        logits  = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits  = K.reshape(logits, (x_shape[0], x_shape[1], x_shape[2]))
        ai      = K.exp(logits - K.max(logits, axis=[1,2], keepdims=True))

        att_weights    = ai / (K.sum(ai, axis=[1,2], keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result         = K.sum(weighted_input, axis=[1,2])
        return result

**Example Using Convolutional Block Attention Module(CBAM)**

In [ ]:
def build_model():
    inp = tf.keras.layers.Input(shape=(256,256,3))

    res_model = resnet18(include_top=False, weights='imagenet', input_shape=(256,256,3))

    ## Freezing Layers
    for layer in res_model.layers[:len(res_model.layers)//10]:
        layer.trainable = False

    # Output
    x = res_model(inp)
    x1 = CAM()(x)
    x1 = layers.Multiply()([x,x1])
    x2 = SAM()(x1)
    x = layers.Multiply()([x1,x2])
    x = AttentionWeightedAverage2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(len(TARGET), activation='softmax', dtype='float32')(x)

    # Compile
    model = tf.keras.Model(inputs=inp, outputs=x)

    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    opt = tf.keras.optimizers.Adam(learning_rate=1e-3)

    model.compile(loss=loss, optimizer=opt, metrics=['accuracy', f1_score])

    return model